In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
try: sys.path.index('/ml.dev/bin')
except: sys.path.append('/ml.dev/bin')

import numpy as n
import pandas as p
#mport Quandl as q
from Quandl import Quandl as q
import datetime as dd
from qoreliquid import *
from matplotlib import pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 20, 5

In [ ]:
# readable datetime format
def qoreGetTime(dt):
    """
    dt = dd.datetime.now()
    """
    if type(dt) != type(dd.datetime(2000,1,1)):
        raise(TypeError("descriptor 'strftime' requires a 'datetime.date' object but received a '{0}'".format(type(dt))))
    
    return dd.datetime.strftime(dt, '%Y-%m-%d %T.%fTGMT-3')
print qoreGetTime(dd.datetime.now())
#print qoreGetTime('')
print time.time()
print type(dd.datetime.now())
print type(dd)

In [ ]:
# scrape stocktwits trending stocks list
# fetch historical prices of listed stocks
#  - google data via quandl
# analyze historical data of listed stocks
#  - normalized, sigmoid filters

import ujson as j
import pandas as p

def parseSTJson(j3):
    lst = []
    #for i in j3['industries']:
    for i in j3['stocks']:
        #print p.DataFrame(i).transpose()
        tdf = p.DataFrame(i)
        lst.append(n.array(tdf.ix[['change'],tdf.columns].get_values()[0], dtype=str))
        #print p.pivot(p.DataFrame(i).ix[['change'],['symbol', 'values']], index='symbol', columns=['change'], values=['values'])
        #print p.pivot(p.DataFrame(i), index='change', columns=['symbol', 'name'], values=['values'])
        #break
        #print
    res2 = p.DataFrame(lst, columns=tdf.columns)
    return res2

def gettickerSymbol(s):
    hque = fetchURL('https://www.google.com/finance?q={0}'.format(s), mode='')
    try:
        hque = re.match(re.compile(r'.*\(([\w]+?:[\w]+?)\).*', re.S), hque).groups()[0]
        return hque
    except:
        return ''

def queryGoogleFinanceForTicker(st1):
    st1
    ty = []
    for i in st1:
        ty.append("{0}".format(gettickerSymbol(i)))
    print ty
    tt = ";".join(ty)
    tt = re.sub(re.compile(r':', re.S), '_', tt).split(';')
    print tt
    ty = []
    for i in tt:
        ty.append("GOOG/{0}".format(i))
    st1 = ty
    return st1

def fromGoogleFinanceChartGetTickers(url):
    """
    url = 'https://www.google.com/finance?chdnp=1&chdd=1&chds=1&chdv=1&chvs=Linear&chdeh=1&chfdeh=1&chdet=1426072726443&chddm=963&chls=IntervalBasedLine&cmpto=NYSE:BITA;NYSE:PAY;NASDAQ:GERN;NYSEMKT:ROX;NYSE:PNK&cmptdms=0;0;0;0;0&q=NASDAQ:URBN&ntsp=0&ei=ux8AVZGVE8ei8waB2IHIDw'
    """
    t1 = re.match(re.compile(r'.*cmpto=.*?q=(.*?)&.*', re.S), url).groups()[0]
    t2 = re.match(re.compile(r'.*cmpto=(.*?)&.*', re.S), url).groups()[0]
    tt = "{0};{1}".format(t1,t2)
    tt = re.sub(re.compile(r':', re.S), '_', tt).split(';')
    ty = []
    for i in tt:
        ty.append("GOOG/{0}".format(i))
    return ty

def jsonValidator():
    #from jsonschema import Draft3Validator
    #my_schema = j.loads(r3) #or however else you end up with a dict of the schema
    #print my_schema
    #Draft3Validator.check_schema(schema)
    #print r3
    ''

#st1 = fromGoogleFinanceChartGetTickers()
#print st1

#url = 'http://stocktwits.com/signals/stocks/week'
url = 'http://stocktwits.com/signals/stocks/day'
html = fetchURL(url, mode='')
res = re.match(re.compile(r'.*?parseJSON\("(.*})"\).*', re.S), html).groups()
r3 = res[0].decode('string_escape')
# todo: save json to log file
j3 = j.loads(r3)

res2 = parseSTJson(j3)
#print res2.ix[0:10,['symbol', 'values']]
print res2.ix[0:10,:]
st1 = list(res2.ix[0:10,['symbol']].transpose().get_values()[0])
print st1
st1 = queryGoogleFinanceForTicker(st1)
print st1
st1 = "\n".join(st1)
st1 = st1.strip().split('\n')

non1 = getDataFromQuandl(st1, plot=False, style='-', columns='Close', tail=100)
non = non1
non = normalizeme(non)
non = sigmoidme(non)
#non.ix[:,['Ratio', 'Bullish', 'Bearish']].plot(style='-'); show();
non.plot(style='-'); legend(non.columns, 2); show();
print non.tail(3).transpose()
non.to_csv('stocktwits-signal.csv', index=None)

In [ ]:
import pandas
import numpy
import requests
from Quandl import Quandl as q

index = ['Dec 12 2296', 'Dec 21 1998', 'Oct 9 2000', 'Oct 19 2001', 'Oct 30 2003', 'Nov 12 2003']
data = pandas.DataFrame(numpy.random.randn(6, 3), index=index, columns=['D', 'B', 'C'])
q.push(data, code='F32C', name='Test', desc='test', authtoken='WVsyCxwHeYZZyhf5RHs2')

In [ ]:
#import requests
q.push(non, code='TESTCODE12321', name='NON', desc='test', authtoken="WVsyCxwHeYZZyhf5RHs2")
#Quandl.push(data, code='F32C', name='Test', desc='test', authtoken='xxxxxx')
#q.get('ALEXA/PSYCH_CENTRAL', authtoken="WVsyCxwHeYZZyhf5RHs2")
#non.

In [ ]:
# visualize stocktwits result
na1 = n.array(res2.ix[:,'values'].get_values(), dtype=float)
na1 = p.DataFrame(na1).ix[0:4,:]

# select and calculate indeces based on 
# available tickers (some brokers may not list all tickers in this list)
#availableTickers = [1] #etoro
availableTickers = [1,2,12,13,19,21,25,27,28,] #plu500
na1 = p.DataFrame(na1).ix[availableTickers,:]

print len(na1)
#print len(na2)
#print p.DataFrame(n.abs(na1))
na1 = n.abs(na1)
print  n.sum(na1)
res2['alloc'] = na1 / n.sum(na1) * 100
#res2['alloc'] = 0
print n.sum(res2.ix[:,'alloc'])
print " ".join(list(res2.ix[0:50,'symbol'].get_values()))
print res2

In [ ]:
htmlp500 = fetchURL('http://www.plus500.com/AllInstruments/AllInstruments.aspx', mode='')
# $x('//*[@id="main"]/div/div[1]/div[2]/div/div/table/tbody/tr/td[1]').forEach(function(e) {console.log(e.innerHTML)});

In [ ]:
non = getDataFromQuandl('PSYCH/EURUSD_R', plot=True, style='.')
non = normalizeme(non)
non = sigmoidme(non)
non.ix[:,['Ratio', 'Bullish', 'Bearish']].plot(style='-'); show();

In [ ]:
WORLDBANK/URY_FR_INR_DPST

In [ ]:
searchQuandl('yield')

In [ ]:
ls -lS '/ml.dev/bin/data/oanda/datafeed/'

In [ ]:
import oandapy
import pandas as p

co = p.read_csv('config.csv', header=None)

env2=co.ix[1,1]
access_token2=co.ix[1,2]
oanda2 = oandapy.API(environment=env2, access_token=access_token2)

acc = oanda2.get_accounts()['accounts']
accid = acc[0]['accountId']
print 'using account: {0}'.format(accid)

response = oanda2.get_prices(instruments="USD_MXN")
prices = response.get("prices")
asking_price = prices[0].get("ask")
print asking_price

In [ ]:
import plotly.graph_objs as gos

In [ ]:
# view real-time plot
import plotly.plotly as py
from plotly.graph_objs import *
# auto sign-in with credentials or use py.sign_in()
py.sign_in('cilixian', 'ks48f6mysz')
trace1 = Scatter(
    x=[], 
    y=[], 
    #stream=dict(token='my_stream_id')
    stream=dict(token='dlun5nb9sr')
)
data = Data([trace1])
py.plot(data)
#s = py.Stream('dlun5nb9sr')
#s.open()
#s.write(dict(x=1, y=2))
#s.close()

In [ ]:
s = py.Stream('dlun5nb9sr')
s.open()
s.write(dict(x=1, y=2))
s.close()

In [ ]:
# source: http://stackoverflow.com/questions/18722196/how-to-set-utc-offset-for-datetime
import datetime as dd
import time
import dateutil.tz
dt = dateutil.parser.parse('2015-03-09T23:08:16.854172Z +0000')
print (dt - dd.datetime(1970,1,1, tzinfo=dateutil.tz.tzoffset('UTC', 0))).total_seconds()  / dd.timedelta(seconds=1).total_seconds()
print dt

In [ ]:
pair = 'EUR_USD'

from tailf import tailf
ticks = 0
for line in tailf('/ml.dev/bin/data/oanda/datafeed/{0}.csv'.format(pair)):
    ticks += 1
    i = line.split(',')
    #print i[4]
    print i
    #print "{0}".format(dd.datetime.strptime(i[3], '%Y-%m-%dT%H:%M:%S.%fZ'))
        
    #if ticks >= 10: break

In [ ]:
from tailf import tailf
s = py.Stream('dlun5nb9sr')
s.open()
"""
for c in range(100):
    #time.sleep(1)
    #print c
    i = n.random.randn(1,2)[0];
    s.write(dict(x=i[0], y=i[1]))
"""
for line in tailf('/ml.dev/bin/data/oanda/datafeed/GBP_JPY.csv'):
    i = line.split(',')
    s.write(dict(x=i[1], y=i[2]))
s.close()

In [ ]:
an1 = p.read_csv('/ml.dev/bin/data/oanda/datafeed/USD_MXN.csv', header=None)
an2 = p.read_csv('/ml.dev/bin/data/oanda/datafeed/USD_SEK.csv', header=None)

an1.plot(legend=None)
an2.plot(legend=None)


In [ ]:
def rmo():
    #"""
    an1 = p.read_csv('/ml.dev/bin/data/oanda/datafeed/USD_MXN.csv', header=None)
    
    an1[4] = 1
    an1[5] = '1'
    print an1.ix[0,3]
    for i in range(0,len(an1)-1):
        #an1.ix[i,5] = dd.datetime.strptime(an1.ix[i,3], '%Y-%m-%dT%H:%M:%S.%fZ')
        ''
        #break
    print an1.ix[0:2,:]
    
    #an1 = p.pivot_table(an1, values=[1,2], index=[3], columns=[0])
    anp = p.pivot_table(an1, values=[4], index=[1], columns=[2], aggfunc=n.sum).fillna(0)
    #an2 = p.read_csv('/ml.dev/bin/data/oanda/datafeed/EUR_USD.csv', header=None)
    #an = an.set_index(3)
    #print an1
    #an.ix[:,[1,2]].plot(style='.'); show();
    #"""
    #print (p.DataFrame(n.sum(anp.ix[:,anp.columns[0:100]]), index=anp.columns[0:100]).plot(legend=False));
    #print list(anp.index)
    #show();
#%prun rmo()
rmo()


In [ ]:
an = p.read_csv('/ml.dev/bin/data/oanda/datafeed/EUR_USD.csv', header=None)
#an = an.set_index(3)
#print an
an.ix[:,[1,2]].plot(style='.'); show();

In [ ]:
import numpy as n

data = [float(val) for val in '4 4'.split(' ')]
print data

In [ ]:
import sys, serial
import numpy as np
from time import sleep
from collections import deque
from matplotlib import pyplot as plt

# plot class
class AnalogPlot:
  # constr
  def __init__(self, analogData):
    # set plot to animated
    self.fig, axplt = plt.subplots() #plt.ion() 
    self.axline, = axplt.plot(analogData.ax)
    self.ayline, = axplt.plot(analogData.ay)
    plt.ylim([0, 1023])

  # update plot
  def update(self, analogData):
    self.axline.set_ydata(analogData.ax)
    self.ayline.set_ydata(analogData.ay)
    self.fig.canvas.draw()
    self.fig.canvas.flush_events()
    #plt.draw()

# class that holds analog data for N samples
class AnalogData:
  # constr
  def __init__(self, maxLen):
    self.ax = deque([0.0]*maxLen)
    self.ay = deque([0.0]*maxLen)
    self.maxLen = maxLen

  # ring buffer
  def addToBuf(self, buf, val):
    if len(buf) < self.maxLen:
      buf.append(val)
    else:
      buf.pop()
      buf.appendleft(val)

  # add data
  def add(self, data):
    assert(len(data) == 2)
    self.addToBuf(self.ax, data[0])
    self.addToBuf(self.ay, data[1])
    
# plot parameters
analogData = AnalogData(100)
analogPlot = AnalogPlot(analogData)
print('plotting data...')

while True:
        #try:
        #.strip(b'\x00').decode('ascii') added for IPython3
        #line = ser.readline().strip(b'\x00').decode('ascii')
        line = '600 400'
        data = [float(val) for val in line.split()]
        
        if(len(data) == 2):
            analogData.add(data)
            analogPlot.update(analogData)
        #except: #Interrupt the kernal to exit
        #    print('Exiting')
        #    break


In [ ]:
%matplotlib inline
import time
import pylab as pl
from IPython import display
for i in range(10):
    try:
        pl.plot(pl.randn(100))
        display.clear_output()
        display.display(pl.gcf())
        pl.gcf().show();
        time.sleep(1.0)
    except:
        ''

In [ ]:
from IPython.display import display, clear_output
import threading
import time
import numpy as n
import pandas as p
from tailf import tailf

tiks = """/ml.dev/bin/data/oanda/datafeed/AUD_CAD.csv  /ml.dev/bin/data/oanda/datafeed/EUR_PLN.csv  /ml.dev/bin/data/oanda/datafeed/SGD_JPY.csv
/ml.dev/bin/data/oanda/datafeed/AUD_CHF.csv  /ml.dev/bin/data/oanda/datafeed/EUR_SEK.csv  /ml.dev/bin/data/oanda/datafeed/TRY_JPY.csv
/ml.dev/bin/data/oanda/datafeed/AUD_HKD.csv  /ml.dev/bin/data/oanda/datafeed/EUR_SGD.csv  /ml.dev/bin/data/oanda/datafeed/USD_CAD.csv
/ml.dev/bin/data/oanda/datafeed/AUD_JPY.csv  /ml.dev/bin/data/oanda/datafeed/EUR_TRY.csv  /ml.dev/bin/data/oanda/datafeed/USD_CHF.csv
/ml.dev/bin/data/oanda/datafeed/AUD_NZD.csv  /ml.dev/bin/data/oanda/datafeed/EUR_USD.csv  /ml.dev/bin/data/oanda/datafeed/USD_CNH.csv
/ml.dev/bin/data/oanda/datafeed/AUD_SGD.csv  /ml.dev/bin/data/oanda/datafeed/EUR_ZAR.csv  /ml.dev/bin/data/oanda/datafeed/USD_CNY.csv
/ml.dev/bin/data/oanda/datafeed/AUD_USD.csv  /ml.dev/bin/data/oanda/datafeed/GBP_AUD.csv  /ml.dev/bin/data/oanda/datafeed/USD_CZK.csv
/ml.dev/bin/data/oanda/datafeed/CAD_CHF.csv  /ml.dev/bin/data/oanda/datafeed/GBP_CAD.csv  /ml.dev/bin/data/oanda/datafeed/USD_DKK.csv
/ml.dev/bin/data/oanda/datafeed/CAD_HKD.csv  /ml.dev/bin/data/oanda/datafeed/GBP_CHF.csv  /ml.dev/bin/data/oanda/datafeed/USD_HKD.csv
/ml.dev/bin/data/oanda/datafeed/CAD_JPY.csv  /ml.dev/bin/data/oanda/datafeed/GBP_HKD.csv  /ml.dev/bin/data/oanda/datafeed/USD_HUF.csv
/ml.dev/bin/data/oanda/datafeed/CAD_SGD.csv  /ml.dev/bin/data/oanda/datafeed/GBP_JPY.csv  /ml.dev/bin/data/oanda/datafeed/USD_INR.csv
/ml.dev/bin/data/oanda/datafeed/CHF_HKD.csv  /ml.dev/bin/data/oanda/datafeed/GBP_NZD.csv  /ml.dev/bin/data/oanda/datafeed/USD_JPY.csv
/ml.dev/bin/data/oanda/datafeed/CHF_JPY.csv  /ml.dev/bin/data/oanda/datafeed/GBP_PLN.csv  /ml.dev/bin/data/oanda/datafeed/USD_MXN.csv
/ml.dev/bin/data/oanda/datafeed/CHF_ZAR.csv  /ml.dev/bin/data/oanda/datafeed/GBP_SGD.csv  /ml.dev/bin/data/oanda/datafeed/USD_NOK.csv
/ml.dev/bin/data/oanda/datafeed/EUR_AUD.csv  /ml.dev/bin/data/oanda/datafeed/GBP_USD.csv  /ml.dev/bin/data/oanda/datafeed/USD_PLN.csv
/ml.dev/bin/data/oanda/datafeed/EUR_CAD.csv  /ml.dev/bin/data/oanda/datafeed/GBP_ZAR.csv  /ml.dev/bin/data/oanda/datafeed/USD_SAR.csv
/ml.dev/bin/data/oanda/datafeed/EUR_CHF.csv  /ml.dev/bin/data/oanda/datafeed/HKD_JPY.csv  /ml.dev/bin/data/oanda/datafeed/USD_SEK.csv
/ml.dev/bin/data/oanda/datafeed/EUR_CZK.csv  /ml.dev/bin/data/oanda/datafeed/NZD_CAD.csv  /ml.dev/bin/data/oanda/datafeed/USD_SGD.csv
/ml.dev/bin/data/oanda/datafeed/EUR_DKK.csv  /ml.dev/bin/data/oanda/datafeed/NZD_CHF.csv  /ml.dev/bin/data/oanda/datafeed/USD_THB.csv
/ml.dev/bin/data/oanda/datafeed/EUR_GBP.csv  /ml.dev/bin/data/oanda/datafeed/NZD_HKD.csv  /ml.dev/bin/data/oanda/datafeed/USD_TRY.csv
/ml.dev/bin/data/oanda/datafeed/EUR_HKD.csv  /ml.dev/bin/data/oanda/datafeed/NZD_JPY.csv  /ml.dev/bin/data/oanda/datafeed/USD_TWD.csv
/ml.dev/bin/data/oanda/datafeed/EUR_HUF.csv  /ml.dev/bin/data/oanda/datafeed/NZD_SGD.csv  /ml.dev/bin/data/oanda/datafeed/USD_ZAR.csv
/ml.dev/bin/data/oanda/datafeed/EUR_JPY.csv  /ml.dev/bin/data/oanda/datafeed/NZD_USD.csv  /ml.dev/bin/data/oanda/datafeed/ZAR_JPY.csv
/ml.dev/bin/data/oanda/datafeed/EUR_NOK.csv  /ml.dev/bin/data/oanda/datafeed/SGD_CHF.csv
/ml.dev/bin/data/oanda/datafeed/EUR_NZD.csv  /ml.dev/bin/data/oanda/datafeed/SGD_HKD.csv"""
tk = tiks.split('  ')

"""
t = {}
for i in xrange(1):
    #time.sleep(5)
    #clear_output()
        #sys.stdout.write('test' + '\n')
    
    t[0] = threading.Thread(target=testThr, args=[tk[i]])
    t[0].daemon = False
    t[0].start()
    
    #sys.stdout.flush()
"""

def groupby(lns, a, b):
    df = p.DataFrame(lns)
    df = df.convert_objects(convert_numeric=True)
    dfg = df.groupby(a)
    #print dfg.describe()
    print dfg[b].agg([n.std,n.mean])

def testThr(fname):
    #for i in xrange(10):
        
    lns = []
    ticks = 0
    
    #plotly stream
    #s = py.Stream('dlun5nb9sr')
    #s.open()
    
    for line in tailf(fname):
        ticks += 1
        
        i = line.split(',')
        #print i[4]
        #print i
        lns.append(i)
        
        time.sleep(0.3)
        clear_output()

        groupby(lns, 1,2)

        #i = line.split(',')
        #s.write(dict(x=i[1], y=i[2]))
        
        sys.stdout.flush()
            
        #if ticks >= 10: break
        
    s.close()
        
testThr('/ml.dev/bin/data/oanda/datafeed/EUR_USD.csv')

In [ ]:
import pandas as p
import numpy as n
from pylab import *

def prepTks(tks):
    tkd = []
    for i in filter(None, tks.split(' ')):
        i = filter(None, i.split('\n'))
        i[0] = i[0].replace('.csv','')
        tkd.append(" ".join(i))
    #print tkd
    #print " ".join(tkd)
    return tkd

def genGrid(tks):
    df = p.DataFrame()
    tkd = prepTks(tks)
    for i in xrange(len(tkd)):
        df = df.combine_first(p.DataFrame(tkd[i].split('_'), index=[0,1], columns=[i]).transpose())
    pairs = p.DataFrame(list(df.ix[:,0].drop_duplicates()) + list(df.ix[:,1].drop_duplicates())).transpose()
    pairs = list(pairs.get_values()[0])
    lp = len(pairs)
    grid = p.DataFrame(n.zeros(n.power(lp,2)).reshape(lp,lp), index=pairs, columns=pairs)
    return grid


In [ ]:
qq = Qoreliquid()

In [ ]:
#####
# Visualization
###
tks = """AUD_CAD.csv  AUD_USD.csv  CHF_JPY.csv  EUR_DKK.csv  EUR_NZD.csv  EUR_ZAR.csv  GBP_NZD.csv  NZD_CAD.csv  SGD_CHF.csv  USD_CNH.csv  USD_INR.csv  USD_SEK.csv  ZAR_JPY.csv  AUD_CHF.csv  CAD_CHF.csv  CHF_ZAR.csv  EUR_GBP.csv  EUR_PLN.csv  GBP_AUD.csv   GBP_PLN.csv  NZD_CHF.csv  SGD_HKD.csv  USD_CNY.csv  USD_JPY.csv  USD_SGD.csv AUD_HKD.csv  CAD_HKD.csv  EUR_AUD.csv  EUR_HKD.csv  EUR_SEK.csv  GBP_CAD.csv   GBP_SGD.csv  NZD_HKD.csv  SGD_JPY.csv  USD_CZK.csv  USD_MXN.csv  USD_THB.csv AUD_JPY.csv  CAD_JPY.csv  EUR_CAD.csv  EUR_HUF.csv  EUR_SGD.csv  GBP_CHF.csv   GBP_USD.csv  NZD_JPY.csv  TRY_JPY.csv  USD_DKK.csv  USD_NOK.csv  USD_TRY.csv AUD_NZD.csv  CAD_SGD.csv  EUR_CHF.csv  EUR_JPY.csv  EUR_TRY.csv  GBP_HKD.csv   GBP_ZAR.csv  NZD_SGD.csv  USD_CAD.csv  USD_HKD.csv  USD_PLN.csv  USD_TWD.csv  AUD_SGD.csv  CHF_HKD.csv  EUR_CZK.csv  EUR_NOK.csv  EUR_USD.csv  GBP_JPY.csv   HKD_JPY.csv  NZD_USD.csv  USD_CHF.csv  USD_HUF.csv  USD_SAR.csv  USD_ZAR.csv"""

#%timeit -n 20 genGrid(tks)
gr = genGrid(tks)

# source: http://matplotlib.org/examples/pylab_examples/hist2d_demo.html
x = randn(len(gr))
y = randn(len(gr))

#normal distribution center at x=0 and y=5
print 
hist2d(x,y,bins=40)
show()


In [ ]:
def readCompanies(fname):
    fp = open(fname,'r')
    #js = j.loads(fp.read())
    df = p.DataFrame()
    i = 0
    for line in fp.readlines():
        js = j.loads(line.split('\n')[0])
        df = df.combine_first(p.DataFrame(js, index=[i]))
        i += 1
    fp.close()
    return df
df = readCompanies('/mldev/lib/bloomberg/bloomberg/backups/companies_products.json')
#df2 = readCompanies('/mldev/lib/bloomberg/bloomberg/companies_products.json')
#df = readCompanies('/mldev/lib/bloomberg/CrawlPubComp/companies.json')http://127.0.0.1:8889/21fad5ec-c1c0-4910-b80b-3b01cd3551e3#


print len(df)
print len(df2)

print df2.set_index('ticker')

df = p.DataFrame()
for i in xrange(len(js)):
    #df = df.combine_first(p.DataFrame(js[i], index=[i]))
    print js[i]
#print df

fp = open('/mldev/lib/stocktwits/tmpscrapy/CrawlPubComp/companies.json', 'r')
js = fp.read()
js = j.loads(js)
len(js)

In [ ]:
import subprocess
def countLines(fname):
    cmd = "wc -l '{0}'".format(fname)
    print cmd
    %timeit -n 1 print int(subprocess.check_output(cmd).split()[0])
    
    %timeit -n 1 print sum(1 for line in open(fname))
print countLines('/data-oanda-datafeed/EUR_USD.csv')
print countLines('/data-oanda-datafeed/GBP_USD.csv')


In [ ]:
import mmap
mmap

In [ ]:
fname = '/data-oanda-datafeed/EUR_USD.csv'
fname = os.path.abspath(fname)
print fname
fname = os.path.dirname(fname)
print fname
#fname = os.chdir(fname)
#print fname
#fname = os.chdir(os.path.dirname(os.path.abspath(fname)))
#print fname

cmd = "wc -l '{0}'".format(fname)
#cmd = "ls"
print cmd
#%timeit -n 1 print
print int(subprocess.check_output(cmd).split()[0])

In [ ]:
import numpy as n
fname = '/data-oanda-datafeed/EUR_USD.csv'
print n.genfromtxt(fname, delimiter=',')

In [ ]:
print p.read_csv(fname, skiprows=40000)

In [ ]:
print 34967750.0/539981
print int(floor(34967750.0/64))

In [ ]:
from pandasdmx import Request
import matplotlib.pyplot as plt
rcParams['figure.figsize'] = 200, 50

class ECB:
    
    def __init__(self, verbose=False):

        self.ecb = Request('ECB')
        cat_resp = self.ecb.get(resource_type = 'categoryscheme')
        if verbose == True: print type(cat_resp)
        #Out[6]: pandasdmx.api.Response
        
        cat_msg = cat_resp.msg
        if verbose == True: print type(cat_msg)
        #Out[8]: pandasdmx.model.StructureMessage
        
        cat_header = cat_msg.header
        if verbose == True: print type(cat_header)
        #Out[10]: pandasdmx.model.Header
        
        categorisations = cat_msg.categorisations
        if verbose == True: print type(categorisations)
        #Out[12]: pandasdmx.model.Categorisations
        
        # part II
        cs = cat_msg.categoryschemes
        
        type(cs)
        #Out[17]: pandasdmx.utils.DictLike
        
        list(cs.keys())
        #Out[18]: ['MOBILE_NAVI']
        
        # part III
        cs0 = cs.aslist()[0]
        for i in cs0.keys():
            if verbose == True: print cs0[i]
        if verbose == True: print 
        if verbose == True: print cs0['03'].name
        #for i in cat_msg.categorisations['03']: print i
        if verbose == True: print cat_msg.dataflows.find('interest')
                
        # part IV
        flows = self.ecb.get(resource_type = 'dataflow')
        flows.msg.dataflows
        #data_resp = self.ecb.get(resource_type = 'data', resource_id = 'MIR', key = '.USD+JPY+GBP...', params = dict(startPeriod = '2014'))
        
        # part VI
        cs0 = cs.aslist()[0]
        
        if verbose == True: print type(cs0)
        #Out[20]: pandasdmx.model.CategoryScheme
        
        # Print the number of categories
        if verbose == True: print len(cs0)
        #Out[21]: 11
        
        # Print ID's of categories
        if verbose == True: print list(cs0.keys())
        #Out[22]: ['04', '08', '09', '06', '03', '01', '10', '02', '11', '07', '05']
        
        # English name of category '07'
        if verbose == True: print cs0['07'].name.en
        #Out[23]: 'Exchange rates'

        # part VII
        cat07_l = cat_msg.categorisations['07']

        if verbose == True: print list(cat_msg.dataflows[i.artefact.id] for i in cat07_l)
        #Out[25]: 
        #[DataflowDefinition : EXR  : Exchange Rates,
        # DataflowDefinition : EXR  : Exchange Rates,
        # DataflowDefinition : WTS  : Trade weights,
        # DataflowDefinition : WTS  : Trade weights]        
    
        # part VIII
        #cat_msg.dataflows.find('rates')
        if verbose == True: print cat_msg.dataflows.find('interest')
        if verbose == True: print cat_msg.dataflows.find('yield')
        #Out[26]: 
        #{'EXR': DataflowDefinition : EXR  : Exchange Rates,
        # 'RIR': DataflowDefinition : RIR  : Retail Interest Rates}
    
    def getDimensions(self, cat_msg, dsd_id):
        exec('dsd_id = cat_msg.dataflows.{0}.structure.id'.format(dsd_id))
        #dsd_id = cat_msg.dataflows.'EXR'.structure.id
        refs = dict(references = 'all')
        refs
        dsd_resp = self.ecb.get(resource_type = 'datastructure', resource_id = dsd_id, params = refs)
        dsd_resp
        dsd = dsd_resp.msg.datastructures[dsd_id]
        dsd
        
        print list(d.id for d in dsd.dimensions.aslist())
        #Out[32]: ['FREQ', 'CURRENCY', 'CURRENCY_DENOM', 'EXR_TYPE', 'EXR_SUFFIX', 'TIME_PERIOD']
        
        return dsd
        
    def getCodeList(self, dsd3):
        for i in dsd3.dimensions.keys():
            print
            print i    
        
            cmd = 'ccl = dsd3.dimensions.{0}.local_repr.enum'.format(i)
            print cmd
            exec(cmd)
            print ccl
            #print ccl.USD, currency_codelist.JPY
            #print dir(ccl)
            try:    
                print len(ccl)
                for j in ccl.aslist():                                                                                                                          
                    print j
            except: ''
            print 
            print 
            
            #currency_codelist = dsd.dimensions.CURRENCY.local_repr.enum
            
            #print len(currency_codelist)
            #Out[34]: 348
            
            #print currency_codelist.USD, currency_codelist.JPY
            #Out[35]: (Code : USD  : US dollar, Code : JPY  : Japanese yen)
            
    def getSeries(self):
        # part V
        data_resp = self.ecb.get(resource_type = 'data', resource_id = 'EXR', key = '.USD+JPY+GBP...', params = dict(startPeriod = '2014'))
        data = data_resp.msg.data
        for i in data.series:
            print i.key
        
        #daily = (s for s in data.series if s.key.FREQ == 'D')
        #cur_df = data_resp.write(daily)
        #print cur_df.shape
        #print cur_df.tail()
        #print cur_df

    def gee(self, nnn):
        self.pa = getDatasetEUR(returnPairs=False)
        self.pa = self.pa.split(' ')
        li = []
        for i in self.pa[0:nnn]:
            li.append(i[3:])
        self.pa = '+'.join(li)
        print self.pa
        
        data_resp = self.ecb.get(resource_type = 'data', resource_id = 'EXR', key = '.{0}...'.format(self.pa), params = dict(startPeriod = '2013-01-01'))
        data = data_resp.msg.data
        daily = (s for s in data.series if s.key.FREQ == 'D')
        cur_df = data_resp.write(daily)
        #print cur_df.shape
        #print cur_df.tail()
        #print cur_df
        #Out[48]: 
        #FREQ                 D        
        #CURRENCY           JPY     USD
        #CURRENCY_DENOM     EUR     EUR
        #EXR_TYPE          SP00    SP00
        #EXR_SUFFIX           A       A
        #2015-05-19      134.36  1.1180
        #2015-05-20      134.43  1.1118
        #2015-05-21      134.79  1.1133
        
        df = cur_df
        df = df.fillna(0)
        df = normalizeme(df)
        df = sigmoidme(df)
        #print df
        #df.plot(legend=False); show();
        #savefig('temp-{0}.png'.format(nnn), dpi=fig.dpi)
        
        ax = df.plot(legend=False);  # s is an instance of Series
        fig = ax.get_figure()
        mkdir_p('plots')
        fig.savefig('plots/EUR_{0}.png'.format(nnn))
        
        #fig = plt.figure()
        #plt.plot(list(cur_df.index), df); show();
        #fig.savefig('temp-{0}.png'.format(nnn), dpi=fig.dpi)

    def getEUR(self):
        
        #for i in xrange(1, len(getDatasetEUR(returnPairs=False).split(' '))-1): 
        #    self.gee(i)
        self.gee(167)

ee = ECB()
#dsd2 = ee.getDimensions(cat_msg, 'EXR')
#dsd2 = ee.getDimensions(cat_msg, 'MIR')
#ee.getCodeList(dsd2)
#ee.getEUR()

In [ ]:
rcParams['figure.figsize'] = 200, 50
data = p.read_csv('/mldev/bin/data/quandl/BNP.EUR.csv')
df = data
df = df.set_index(df.columns[0]).fillna(0)
#df = df.ix['2013-01-01':,:]
print df.shape
dfn = df.get_values()
dfn = normalizeme(dfn)
dfn = sigmoidme(dfn)
p.DataFrame(dfn, index=df.index, columns=df.columns).plot(legend=False); show();

In [ ]:
ecb = Request('ECB')
#data_resp = ecb.get(resource_type = 'data', resource_id = 'MIR', key = '....', params = dict(startPeriod = '2014'))
data_resp = ecb.get(resource_type = 'data', resource_id = 'EXR', key = '.USD+GBP+JPY...', params = dict(startPeriod = '2000'))

print type(data_resp.msg)
#Out[37]: pandasdmx.model.GenericDataMessage

data = data_resp.msg.data

print type(data)
#Out[39]: pandasdmx.model.GenericDataSet

daily = (s for s in data.series if s.key.FREQ == 'D')

cur_df = data_resp.write(daily)

print cur_df.shape
#Out[47]: (354, 2)

#print cur_df.tail()
cur_df = normalizeme2(cur_df)
print cur_df.plot()
#Out[48]: 
#FREQ                 D        
#CURRENCY           JPY     USD
#CURRENCY_DENOM     EUR     EUR
#EXR_TYPE          SP00    SP00
#EXR_SUFFIX           A       A
#2015-05-19      134.36  1.1180
#2015-05-20      134.43  1.1118
#2015-05-21      134.79  1.1133
#2015-05-22      135.01  1.1164
#2015-05-25      133.39  1.0978

In [ ]:
df = p.read_csv('/home/qore2/Desktop/data.csv')
df.ix[0:len(df),:].convert_objects(convert_numeric=True).fillna(0) #.get_values() #.plot()

In [ ]:
#n.sin(li)
lss = sin(linspace(0,10, num=10000))
lss2 = sin(linspace(0, 100, num=10000))
lss3 = cos(linspace(0, 200, num=10000))
lss4 = sin(linspace(0, 300, num=10000))
plot(lss); show();
#plot(lss2); show();
#plot(lss3); show();
#plot(lss + lss2); show();
plot(lss + lss2 + lss3 + lss4); show();

In [ ]:
import random as rr
import numpy as n
import pandas as p

def t1():
    li = []
    for i in xrange(100):
        x1 = '%.5f' % float(rr.random()*1)
        li.append(float(x1) + 1.5)
    print li
    print p.DataFrame(li).describe()
    plot(li); show();

for i in xrange(10):
    t1()

In [ ]:
def fakeTicker(ticker=False):
    pri = 1.5
    li2 = []
    
    for i in xrange(10000):
    #while True:
        ri = rr.random()/10000
        
        if i % 2 == 0:  il = 1
        else:           il = -1

        pri += ri * il
        li2.append(pri)
        if ticker == True:
            print pri
            time.sleep(rr.random())
    if ticker == True:
        plot(li2)
        print p.DataFrame(li2).describe()
        return li2
    
fakeTicker(ticker=True)

In [ ]:
# real-time account summary
oq = OandaQ()
print oq.oanda2.get_account(oq.aid)['unrealizedPl']

fakeTicker(ticker=True)

try:
    print oq.oanda2.get_positions(oq.aid)['positions'][0]#['units']
    print
    trades = oq.oanda2.get_trades(oq.aid)['trades']
    print p.DataFrame(trades).set_index('id').transpose()#['positions'][0]#['units']
except:
    ''

In [ ]:
oq = OandaQ()

In [ ]:
oq.oanda2.get_positions(oq.aid)

In [ ]:
oq.oanda2.close_position(oq.aid, 'EUR_USD')
oq.oanda2.close_position(oq.aid, 'USD_JPY')
oq.oanda2.close_position(oq.aid, 'AUD_JPY')

In [ ]:
oq.trade(5, 30, 'eu', 's')
#oq.trade(1, 20, 'au', 'b')
#oq.trade(1, 20, 'ej', 'b')

In [ ]:
#oq.trade(1, 40, 'eu', 's')
oq.trade(10, 40, 'au', 's')
#oq.trade(3, 400, 'ej', 's')

In [ ]:
def realtimechart(trend):
    stop = 86
    if trend == 'up':
        oq.trade(1, stop, 'uj', 'b')
        oq.trade(1, stop, 'eu', 's')
    if trend == 'down':
        oq.trade(1, stop, 'uj', 's')
        oq.trade(1, stop, 'eu', 'b')

In [ ]:
realtimechart('up')

In [ ]:
realtimechart('down')

In [ ]:
oq.trade(1, 10, 'uj', 's')
oq.oanda2.create_order()

In [ ]:
oq.sell(1, 10)

In [ ]:
#oq.buy(1, 86) # h1
#oq.buy(1, 25) # 30m
oq.buy(1, 25) # 1m

In [ ]:
# source: https://bearcart.readthedocs.org/en/latest/gettingstarted.html
import bearcart
import pandas as pd

html_path = r'index.html'
data_path = r'data.json'
js_path = r'rickshaw.min.js'
css_path = r'rickshaw.min.css'

#All of the following import code comes from Wes McKinney's book, Python
#for Data Analysis

import pandas.io.data as web

all_data = {}
for ticker in ['AAPL', 'GOOG']:
    all_data[ticker] = web.get_data_yahoo(ticker, '1/1/2014', '1/1/2015')

price = pd.DataFrame({tic: data['Adj Close']
                      for tic, data in all_data.iteritems()})


#all_data = {}
#for ticker in ['AAPL', 'GOOG', 'XOM', 'MSFT', 'INTC', 'YHOO']:
#    all_data[ticker] = web.get_data_yahoo(ticker, '1/1/2014', '1/1/2015')
#price = pd.DataFrame({tic: data['Adj Close'] for tic, data in all_data.iteritems()})

print price
price = price.bfill()
price = normalizeme(price)
price = sigmoidme(price)
#%pylab inline
price.plot()

#vis = bearcart.Chart(price, plt_type='area')
vis = bearcart.Chart(price)
vis = bearcart.Chart(price, hover=False, legend=False)

#df = pd.concat([price['AAPL'], price['GOOG']], axis=1)[:100]
#vis = bearcart.Chart(df, plt_type='scatterplot', colors={'AAPL': '#1d4e69',
#                                                         'GOOG': '#3b98ca' })

#vis.create_chart(html_path=html_path, data_path=data_path, js_path=js_path, css_path=css_path)

# Serving HTTP on 0.0.0.0 port 5556
# http://askubuntu.com/questions/377389/how-to-easily-start-a-webserver-in-any-folder

In [ ]:
def dataviz(df):

    # source: https://bearcart.readthedocs.org/en/latest/gettingstarted.html
    import bearcart
    import pandas as pd
    import pandas.io.data as web

    html_path = r'index.html'
    data_path = r'data.json'
    js_path = r'rickshaw.min.js'
    css_path = r'rickshaw.min.css'

    #All of the following import code comes from Wes McKinney's book, Python
    #for Data Analysis

    %pylab inline
    rcParams['figure.figsize'] = 20, 5

    df = df.bfill().ffill()
    df = normalizeme(df)
    df = sigmoidme(df)
    df.plot()

    #vis = bearcart.Chart(df, plt_type='area')
    #vis = bearcart.Chart(df)
    vis = bearcart.Chart(df, hover=False, legend=False)

    #df = pd.concat([df['AAPL'], df['GOOG']], axis=1)[:100]
    #vis = bearcart.Chart(df, plt_type='scatterplot', colors={'AAPL': '#1d4e69',
    #                                                         'GOOG': '#3b98ca' })

    vis.create_chart(html_path=html_path, data_path=data_path, js_path=js_path, css_path=css_path)
    
    print 'paste into terminal: cd /ml.dev/bin/datafeeds/; python -m SimpleHTTPServer 5556'
    print '         should see: Serving HTTP on 0.0.0.0 port 5556'
    # http://askubuntu.com/questions/377389/how-to-easily-start-a-webserver-in-any-folder

df = p.read_csv('/mldev/lib/crawlers/finance/dataPipeline.scrapy/investingWorldGovernmentBonds_numbeo.csv')

#df.ix[:, 'name	chg	country	currencyCode	period	high	chgpcnt	low	time	prev	iyield'.split('	')]
#df = df.set_index('name').ix[:, 'chg	high	chgpcnt	low	time	prev	iyield'.split('	')]
df = df.ix[:, 'name	chg	high	chgpcnt	low	time	prev	iyield'.split('	')]
#df
df = df.pivot('time', 'name')['iyield'].bfill().ffill()
#dataviz(df)
print df.tail(10)
df = normalizeme(df)
df = sigmoidme(df)
#df.plot()

In [ ]:
import pymongo
# Connection to Mongo DB
try:
    conn = pymongo.MongoClient(port=27017)
    print "Connected successfully!!!"
except pymongo.errors.ConnectionFailure, e:
    print "Could not connect to MongoDB: %s" % e 
    ''
db = conn.numbeo
#db.investingWorldGovernmentBondsfind()#.stats()

In [ ]:
import time, datetime as dd

print datetime.datetime.strptime('19/9'+'/2015', '%d/%m/%Y')
# '%Y-%m-%dT%H:%M:%S.%fZ'
print datetime.datetime.strptime('2015-09-19 '+'08:07:51', '%Y-%m-%d %H:%M:%S')

In [ ]:
[0]*10

In [ ]:
db = conn.numbeo
#db.numbeo.investingWorldGovernmentBonds.find().length()
df = p.DataFrame()
for i in db.investingWorldGovernmentBonds2.find()[0:100]:
    try:
        i['name']
        df = df.combine_first(p.DataFrame(i, index=[i['_id']]))
    except KeyError as e:
        ''
#df = df.ix[:,' chg chgpcnt high iyield low prev'.split(' ')]
df = df.ix[:,'time'.split(' ')]#.get_values()
#df['timei'] = map(lambda x: datetime.datetime.strptime('2015-09-19 '+x, '%Y-%m-%d %H:%M:%S'), df['time'])
#df['timei'] = map(lambda x: re.sub(re.compile(r'.*?([\d]+)\/([\d]+).*'), '\\1', x), df['time'])
df['timei'] = [0]*len(df['time'])
print len(df['time'])
print len(df['timei'])
for i in xrange(len(df['time'])):
        #print i
        stri = df.ix[i, 'time']
        print stri
        df.ix[0, 'timei'] = i
        print i
        #print df.ix[i, 'time']
        #try:
        res = re.match(re.compile(r'.*?(([\d]+)\/([\d]+)).*'), stri).groups(1)[0]
        #print res
        df['timei'][0] = res
        #except Exception as e:
        #    print e
        #    ''
        #print df.ix[i, 'timei']
#df['timei'] = map(lambda x: re.match(re.compile(r'.*?([\d]+)\/([\d]+).*'), x).groups(1), df['time'])
print df#.transpose()
#db.ticks.find()

In [ ]:
df.ix[7,'time'] = None